## Import data from HDFS to MongoDB
---

### Steps:
- Prepare the MongoDB database and collection

```bash
# Use mongo shell to create a database (spark_db) and a collection (books)
mongosh
use spark_db
db.createCollection('books')
```

- Connect to MongoDB using `pymongo`
- Connect to HDFS and read the data using `spark.read.csv`
- Select a subset of the Spark DataFrame to import using `sample` method
- Transform the data into a dictionary using `to_dict` method
- Insert the data into MongoDB using `insert_many` method

In [1]:
# Connect to MongoDB

import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
database = client['spark_db']
books = database['books_hypothesis_2']

In [3]:
# Connect to HDFS

import findspark
findspark.init()
import pyspark

# Initialize Spark Context
spark = pyspark.sql.SparkSession.builder.appName("hypotheses_testing").getOrCreate()
spark.stop()

hypothesis_number = 2
sc = pyspark.SparkContext(appName="hypothesis_"+str(hypothesis_number))

# Create the Spark Session
spark_session = pyspark.sql.SparkSession(sc)

# Load the data
df_joined = spark_session.read.csv("hdfs://localhost:9900/user/book_reviews/books_rating.csv", header=True, inferSchema=True)

23/09/06 11:17:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# Select a random subset of the big data to import
N_to_sample = 200000
df_sample = df_joined.sample(withReplacement = False, fraction = N_to_sample/df_joined.count(), seed = 42)

# Convert to a dictionary
df_sample_dict = df_sample.toPandas().to_dict(orient='records')

# Insert into MongoDB
books.insert_many(df_sample_dict)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=22519Kb max_used=22543Kb free=108553Kb
 bounds [0x000000010a1d8000, 0x000000010b808000, 0x00000001121d8000]
 total_blobs=9472 nmethods=8480 adapters=904
 compilation: disabled (not enough contiguous free space left)
